In [1]:
from   selenium import webdriver
from   selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
import sys
import time
import MySQLdb
import requests
import os
import glob



In [9]:
driver = webdriver.Chrome(r'C:\Users\HIM\Desktop\chromedriver_win32\chromedriver') #driver load
driver.get('http://10.0.1.86/tatoc/advanced/hover/menu') #open website
main_menu = driver.find_element_by_css_selector('body > div > div.page > div.menutop.m2 > span.menutitle') #dropdown menu
sub = driver.find_element_by_css_selector('body > div > div.page > div.menutop.m2 > span:nth-child(5)') #next button
ActionChains(driver).move_to_element(main_menu).click(sub).perform() #perform the action by chaining



### Database query

In [132]:
driver = webdriver.Chrome(r'C:\Users\HIM\Desktop\chromedriver_win32\chromedriver') #driver load
driver.get('http://10.0.1.86/tatoc/advanced/query/gate');
sym = driver.find_element_by_id('symboldisplay').text #get symbol

myConnection = MySQLdb.connect( host="10.0.1.86", user="tatocuser", passwd="tatoc01", db="tatoc" ) #make connection to DB
cur = myConnection.cursor()
cur.execute( "Select id from identity where symbol='{0}'".format(sym)) #query to get ID matching symbol
idd = cur.fetchone() #get id 

cur.execute( "SELECT name,passkey FROM credentials where id='{0}'".format(idd[0]) ) #query to get name and passkey for the ID
name,key = cur.fetchone() #get name and passkey

driver.find_element_by_id('name').send_keys(name) 
driver.find_element_by_id('passkey').send_keys(key)
time.sleep(2)

driver.find_element_by_id('submit').click()

### Rest

In [3]:
# driver.get('http://10.0.1.86/tatoc/advanced/video/player');
driver = webdriver.Chrome(r'C:\Users\HIM\Desktop\chromedriver_win32\chromedriver') #driver load

driver.get('http://10.0.1.86/tatoc/advanced/rest');

idd = driver.find_element_by_id('session_id').text.split(': ')[1]  #get session ID
response = requests.get('http://10.0.1.86/tatoc/advanced/rest/service/token/{0}'.format(idd)) #send GET request with sessio ID
data = response.json() # storing response

#making to dictionary for POST request
d = {
    'id':idd,
    'signature':data['token'],
    'allow_access':1
}

r = requests.post('http://10.0.1.86/tatoc/advanced/rest/service/register/',d) #send POST request
driver.find_element_by_css_selector('a').click()


### Downloading

In [7]:
download_dir = r"C:\Users\HIM\Desktop\chromedriver_win32"
chrome_options = webdriver.ChromeOptions() #settings for chrome

#making dictionary for chrome options
preferences = {"download.default_directory": download_dir ,
                      "directory_upgrade": True,
                      "safebrowsing.enabled": True }

chrome_options.add_experimental_option("prefs", preferences)
driver = webdriver.Chrome(r'C:\Users\HIM\Desktop\chromedriver_win32\chromedriver',chrome_options=chrome_options)#launching chrome with set prefences

driver.get('http://10.0.1.86/tatoc/advanced/file/handle')
driver.find_element_by_css_selector('a').click()

while len(glob.glob(download_dir+'/*.dat')) == 0: #keep cheking until the file is downloaded
    time.sleep(1)

files = glob.glob(download_dir+'/*') #get list of all files available in directory
files.sort(key=os.path.getmtime) #sort them by Time
latest_file = files[-1] #get the latest one from all files

f = open(latest_file,'r') #read file
signature = f.readlines()[2].split(': ')[1] #store signatures
f.close() 

time.sleep(1) #to have a look

driver.find_element_by_id('signature').send_keys(signature)
driver.find_element_by_class_name('submit').click()